In [1]:
using VMLS
using LinearAlgebra

## Chapter 5 

# Linear independence 


### 5.1 Linear dependence 

### 5.2 Basis 

**Cash flow replication.** Let’s consider cash flows over 3 periods, given by 3-vectors. 
We know from VMLS page [93](https://web.stanford.edu/~boyd/vmls/vmls.pdf#section*.108) that the vectors 

$$
e_1 = \begin{bmatrix} 1\\ 0\\ 1\end{bmatrix}, 
l_1 = \begin{bmatrix} 1\\ -(1+r)\\ 0 \end{bmatrix},
l_2 = \begin{bmatrix} 0\\ 1\\ −(1 + r)  \end{bmatrix}
$$

form a basis, where $r$ is the (positive) per-period interest rate. The first vector $e_1$ 
is a single payment of $\$1$ in period (time) t = 1. The second vector $l_1$ is loan of $\$1$ 
in period $t = 1$, paid back in period $t = 2$ with interest $r$. The third vector $l_2$ is 
loan of $\$1$ in period $t = 2$, paid back in period $t = 3$ with interest $r$. Let’s use this 
basis to replicate the cash flow $c = (1, 2,−3)$ as 

$$
c = α_1e_1 + α_2l_1 + α_3l_2 = α_1 \begin{bmatrix} 1 \\ 0 \\ 0\end{bmatrix} + α_2 \begin{bmatrix} 1\\−(1 + r) \\ 0 \end{bmatrix} + α_3 \begin{bmatrix} 0 \\ 1\\  −(1 + r) \end{bmatrix}.
$$

From the third component we have 
$$c_3 = α_3(−(1 + r)), so$$ 
$$α_3 = \frac{−c_3}{(1 + r)}.$$ 
From the second component we have 
$$c_2 = α_2(−(1 + r)) + α_3 = α_2(−(1 + r)) − \frac{c_3}{(1 + r)}, so$$
$$α_2 = \frac{−c_2}{(1 + r)} − \frac{c_3}{(1 + r)^2}.$$
Finally from $c_1 = α_1 + α_2$, we have
$$α_1 = c_1 + \frac{c_2}{(1 + r)} + \frac{c_3}{(1 + r)^2},$$ which is the net present value (NPV) of the cash flow $c$. 

Let’s check this in Julia using an interest rate of $5\%$ per period, and the specific cash flow `c = (1, 2,−3)`. 


**Figure 4.2** Final clustering

### 4.4 Examples

### 4.5 Applications

In [3]:
r = 0.05
e1 = [1,0,0]; l1 = [1,-(1+r),0]; l2 = [0,1,-(1+r)];
c = [1,2,-3];
# Coefficients of expansion
alpha3 = -c[3]/(1+r);
alpha2 = -c[2]/(1+r) -c[3]/(1+r)^2;
alpha1 = c[1] + c[2]/(1+r) + c[3]/(1+r)^2 # NPV of cash flow

[("a1:", alpha1),
("a2:", alpha2), 
("a3:", alpha3)]

3-element Array{Tuple{String,Float64},1}:
 ("a1:", 0.18367346938775508)
 ("a2:", 0.8163265306122449) 
 ("a3:", 2.857142857142857)  

In [4]:
alpha1*e1 + alpha2*l1 + alpha3*l2

3-element Array{Float64,1}:
  1.0
  2.0
 -3.0

(Later in the course we’ll use an automated and simple way to find the coefficients in the expansion of a vector in a basis.)

### 5.3 Orthonormal vectors
**Expansion in an orthonormal basis.** Let’s check that the vectors 
$$
a_1 = \begin{bmatrix} 0\\ 0\\ -1\end{bmatrix}, 
a_2 = \frac{1}{\sqrt{2}}\begin{bmatrix} 1\\ 1\\ 0 \end{bmatrix},
a_3 = \frac{1}{\sqrt{2}}\begin{bmatrix} 1\\ -1\\ 0\end{bmatrix}
$$

form an orthonormal basis, and check the expansion of $x = (1, 2, 3)$ in this basis,

$$x = (a^{T}_{1} x)a_1 + · · ·+ (a^{T}_{n}x)a_n.$$

In [5]:
a1 = [0,0,-1]; a2 = [1,1,0]/sqrt(2); a3 = [1,-1,0]/sqrt(2);
norm(a1), norm(a2), norm(a3)

(1.0, 0.9999999999999999, 0.9999999999999999)

In [6]:
a1'*a2, a1'*a3, a2'*a3

(0.0, 0.0, 0.0)

In [7]:
x = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [8]:
# Get coefficients of x in orthonormal basis
beta1 = a1'*x; beta2 = a2'*x; beta3 = a3'*x
# Expansion of x in basis
xexp = beta1*a1 + beta2*a2 + beta3*a3

3-element Array{Float64,1}:
 0.9999999999999999
 1.9999999999999996
 3.0               

### 5.4 Gram–Schmidt algorithm

The following is a Julia implementation of Algorithm [5.1](https://web.stanford.edu/~boyd/vmls/vmls.pdf#algorithmctr.5.1) in VMLS (Gram–Schmidt algorithm). It takes as input an array `[ a[1], a[2], ..., a[k] ]`, containing the $k$ vectors $a_1, . . . , a_k$. If the vectors are linearly independent, it returns an array `[ q[1], ..., q[k] ]` with the orthonormal set of vectors computed by the Gram – Schmidt algorithm. If the vectors are linearly dependent and the Gram–Schnidt algorithm terminates early in iteration `i`, it returns the array `[ q[1], ..., q[i] ]` of length `i`.

In [10]:
function gram_schmidt(a; tol = 1e-10)
    
    q = []
    for i = 1:length(a)
        qtilde = a[i]
        for j = 1:i-1
            qtilde -= (q[j]'*a[i]) * q[j]
        end
        if norm(qtilde) < tol
            println("Vectors are linearly dependent.")
            return q
        end
        push!(q, qtilde/norm(qtilde))
        end;
    return q
end

gram_schmidt (generic function with 1 method)

On `line 3`, we initialize the output array as the empty array. In each iteration, we add the next vector to the array using the push! function (`line 13`).

**Example.** We apply the function to the example on page [100](https://web.stanford.edu/~boyd/vmls/vmls.pdf#section*.117) of VMLS.

In [11]:
a = [ [-1, 1, -1, 1], [-1, 3, -1, 3], [1, 3, 5, 7] ]

3-element Array{Array{Int64,1},1}:
 [-1, 1, -1, 1]
 [-1, 3, -1, 3]
 [1, 3, 5, 7]  

In [12]:
q = gram_schmidt(a)

3-element Array{Any,1}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]  
 [-0.5, -0.5, 0.5, 0.5]

In [14]:
# test orthnormality
[("norm(q[1]:)",norm(q[1])),
("q[1]'*q[2]:",q[1]'*q[2]),
("q[1]'*q[3]:",q[1]'*q[3]),
("norm(q[2]):",norm(q[2])),
("q[2]'*q[3]:",q[2]'*q[3]),
("norm(q[3]):",norm(q[3]))]

6-element Array{Tuple{String,Float64},1}:
 ("norm(q[1]:)", 1.0)
 ("q[1]'*q[2]:", 0.0)
 ("q[1]'*q[3]:", 0.0)
 ("norm(q[2]):", 1.0)
 ("q[2]'*q[3]:", 0.0)
 ("norm(q[3]):", 1.0)

**Example of early termination.** If we replace $a_3$ with a linear combination of $a_1$, and $a_2$ the set becomes linearly dependent.

In [21]:
b = [ a[1], a[2], 1.3*a[1] + 0.5*a[2] ]

3-element Array{Array{Float64,1},1}:
 [-1.0, 1.0, -1.0, 1.0]
 [-1.0, 3.0, -1.0, 3.0]
 [-1.8, 2.8, -1.8, 2.8]

In [22]:
q = gram_schmidt(b)

Vectors are linearly dependent.


2-element Array{Any,1}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]  

**Example of independence-dimension inequality.** We know that any three $2$ - vectors must be dependent. Let’s use the Gram-Schmidt algorithm to verify this for three specific vectors.',

In [23]:
three_two_vectors = [ [1,1], [1,2], [-1,1] ];
q = gram_schmidt(three_two_vectors)

Vectors are linearly dependent.


2-element Array{Any,1}:
 [0.707107, 0.707107] 
 [-0.707107, 0.707107]